# UCI Catalog Builder
This script scrapes the UCI website to retrieve class information



In [2]:
# imports

from catalog import Catalog
from course import Course

from bs4 import BeautifulSoup
import requests

print('Imports Completed')

Imports Completed


In [3]:
# variable setup
catalog = Catalog()

all_courses_url = 'http://catalogue.uci.edu/allcourses/'
save_file_path = 'catalog.json'

print('Variables Initialized')

Variables Initialized


In [ ]:
from alive_progress import alive_bar

all_courses_page = requests.get(all_courses_url)
all_courses_soup = BeautifulSoup(all_courses_page.content, 'html.parser')

print('URLs Soupified:')

section_soups = []
a_to_z_index = all_courses_soup.find('div', id='atozindex')
with alive_bar(len(a_to_z_index)) as bar:
    for section in a_to_z_index.find_all('a', href=True):
        section_url = all_courses_url + section['href'].split('/')[2]
        # print(section_url)
        section_page = requests.get(section_url)
        section_soups.append(BeautifulSoup(section_page.content, 'html.parser'))
        bar()


print(f'{len(section_soups)} Soups Made')

UnicodeEncodeError: 'charmap' codec can't encode characters in position 1-41: character maps to <undefined>

In [4]:
courses = []
for section_soup in section_soups:
    courses += section_soup.find_all('div', class_='courseblock')

print(f'{len(courses)} Courses Found')

5853 Courses Found


In [5]:
course_blocks = []
for course in courses:
    course_blocks.append((course.find('p', class_='courseblocktitle'), course.find('div', class_='courseblockdesc')))

print('Course Blocks Found')

Course Blocks Found


In [6]:
for course_block_title, course_block_desc in course_blocks:
    course_block_title_text = course_block_title.text.split(sep='.')
    section_code = course_block_title_text[0].split(sep='\xa0')
    section = ' '.join(section_code[:-1])
    code = section_code[-1]
    title = course_block_title_text[1].strip()
    try:
        units = course_block_title_text[2].split()[0]
    except IndexError:
        units = 0

    course_block_desc_children = course_block_desc.findChildren()
    description = course_block_desc_children[0].text

    design_units = 0
    prerequisites = []
    corequisites = []

    for block in course_block_desc_children:
        text = block.text
        if 'Prerequisite:' in text:  # Prerequisite
            if 'Corequisite:' in text:  # Prerequisite and Corequisite
                text = text.split(sep='\n')
                corequisites = text[0].partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(
                    sep=' and ')
                prerequisites = text[1].partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(
                    sep=' and ')
            else:  # just Prerequisite
                prerequisites = text.partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(sep=' and ')
        elif 'Corequisite:' in text:  # just Corequisite
            corequisites = text.partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(sep=' and ')

    def attempt_split(li):
        if len(li) < 2:
            return li
        ret = []
        for el in li:
            if el[0] == '(' and el[-1] == ')':
                ret.append(el[1:-1].split(' or '))
            else:
                ret.append(el)
        return ret

    catalog.add(Course(section, code, title, units, description,
                       attempt_split(corequisites), attempt_split(prerequisites)))

print('Catalog Filled')

Catalog Filled


In [7]:
catalog.save_to_file(save_file_path)

print('Catalog Saved')

Catalog Saved
